In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from Learners import *

### Data

In [2]:
import pandas as pd
df=pd.read_csv('data.csv',index_col=False)
df.head()

,sex,yob,g2000,g2002,g2004,p2000,p2002,p2004,treatment,cluster,voted,hh_id,hh_size,numberofnames,p2004_mean,g2004_mean
0,male,1941,yes,yes,yes,no,yes,No,Civic Duty,1,No,1.0,2,21,0.095238,0.857143
1,female,1947,yes,yes,yes,no,yes,No,Civic Duty,1,No,1.0,2,21,0.095238,0.857143
2,male,1951,yes,yes,yes,no,yes,No,Hawthorne,1,Yes,2.0,3,21,0.047619,0.857143
3,female,1950,yes,yes,yes,no,yes,No,Hawthorne,1,Yes,2.0,3,21,0.047619,0.857143
4,female,1982,yes,yes,yes,no,yes,No,Hawthorne,1,Yes,2.0,3,21,0.047619,0.857143


In [3]:
df.loc[df['sex']=='male', 'sex']=0
df.loc[df['sex']=='female', 'sex']=1
df['age']=2019-df['yob']
for item in ['g2000','g2002','p2000','p2002','p2004','voted']:
    df.loc[df[item]=='yes', item]=1
    df.loc[df[item]=='Yes', item]=1
    df.loc[df[item]=='no', item]=0
    df.loc[df[item]=='No', item]=0

In [4]:
df = df[(df['treatment']==' Control') | (df['treatment']==' Neighbors')]
df.loc[df['treatment']==' Control', 'treatment']=0
df.loc[df['treatment']==' Neighbors', 'treatment']=1

In [5]:
df.head()

,sex,yob,g2000,g2002,g2004,p2000,p2002,p2004,treatment,cluster,voted,hh_id,hh_size,numberofnames,p2004_mean,g2004_mean,age
5,0,1981,0,0,yes,0,0,0,0,1,0,3.0,3,21,0.047619,0.904762,38
6,1,1959,1,1,yes,0,1,0,0,1,1,3.0,3,21,0.047619,0.904762,60
7,0,1956,1,1,yes,0,1,0,0,1,1,3.0,3,21,0.047619,0.904762,63
8,1,1968,0,0,yes,0,1,0,0,1,0,4.0,2,21,0.047619,0.904762,51
9,0,1967,1,1,yes,0,1,0,0,1,0,4.0,2,21,0.047619,0.904762,52


In [6]:
X=np.array(df[['sex','age','g2000','g2002','p2000','p2002','p2004']], dtype=np.int)
y=np.array(df['voted'],dtype=np.int)
treatment=np.array(df['treatment'],dtype=np.int)

In [7]:
print(X[treatment==1].shape,X[treatment==0].shape)

(38201, 7) (191243, 7)


In [15]:
38201/(191243+38201)

0.16649378497585468

### Honest RF

In [15]:
# T
learner_t = BaseTClassifier(learner0=HonestRF(rf=RandomForestClassifier()), learner1=HonestRF(rf=RandomForestClassifier()))
cate_t_honest = learner_t.get_cate(X, y, treatment)
# S
learner_s = BaseSClassifier(learner=HonestRF(rf=RandomForestClassifier()))
cate_s_honest = learner_s.get_cate(X, y, treatment)
# X

In [16]:
cate_t_honest

array([0.12672722, 0.3076601 , 0.0394226 , ..., 0.03954933, 0.05038559,
       0.08570276])

In [17]:
cate_s_honest

array([0.08806366, 0.16150543, 0.03335664, ..., 0.15095236, 0.10993326,
       0.06823671])

In [18]:
np.corrcoef(cate_t_honest, cate_s_honest)

array([[1.       , 0.4359388],
       [0.4359388, 1.       ]])

### RF

In [11]:
# T
learner_t = BaseTClassifier(learner0=RandomForestClassifier(max_leaf_nodes=3000), learner1=RandomForestClassifier(max_leaf_nodes=3000))
cate_t_gt = learner_t.get_cate(X, y, treatment)
# S
learner_s = BaseSClassifier(learner=RandomForestClassifier())
cate_s_gt = learner_s.get_cate(X, y, treatment)
# X

In [12]:
cate_t_gt

array([0.06177048, 0.148895  , 0.05775737, ..., 0.1008212 , 0.06047814,
       0.0527399 ])

In [13]:
cate_s_gt

array([0.06188231, 0.15838654, 0.06089221, ..., 0.09526141, 0.06282674,
       0.04719724])

In [14]:
np.corrcoef(cate_t_gt, cate_s_gt)

array([[1.       , 0.9886924],
       [0.9886924, 1.       ]])

In [9]:
cate_t,cate_s,gt = [],[],[]
X_0, X_1, y_0,y_1 = X[treatment==0], X[treatment==1], y[treatment==0],y[treatment==1]
learner_t_gt = BaseTClassifier(learner0=RandomForestClassifier(max_leaf_nodes=3000), learner1=RandomForestClassifier(max_leaf_nodes=3000))
learner_t_gt.fit(X,y,treatment)
for i in [0.001,0.002,0.005,0.008,0.01,0.02,0.05,0.1,0.2,0.5]:
    len0, len1 = int(i*len(X_0)), int(i*len(X_1))
    ind0 = np.random.choice(len(X_0),len0,replace=False)
    ind1 = np.random.choice(len(X_1),len1,replace=False)
    Xtrain = np.concatenate((X_0[ind0],X_1[ind1]),axis=0)
    ytrain = np.concatenate((y_0[ind0],y_1[ind1]),axis=0)
    Xtest = np.concatenate((X_0[np.delete(np.arange(len(X_0)),ind0)],X_1[np.delete(np.arange(len(X_1)),ind1)]),axis=0)
    gt.append(learner_t_gt.get_cate2(Xtrain,ytrain,Xtest,len0, len1,0))
    learner_t = BaseTClassifier(learner0=RandomForestClassifier(max_leaf_nodes=3000), learner1=RandomForestClassifier(max_leaf_nodes=3000))
    cate_t.append(learner_t.get_cate2(Xtrain,ytrain,Xtest,len0, len1))
    learner_s = BaseSClassifier(learner=RandomForestClassifier())
    cate_s.append(learner_s.get_cate2(Xtrain,ytrain,Xtest,len0, len1))

In [26]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(gt[-1], cate_s[-1], squared=False)

In [27]:
rmse

0.13722164131809483